In [25]:
import os
import sys
import re
import pandas as pd
import nltk
from nltk.corpus import stopwords
print(stopwords.words('english'))

# nltk.download('stopwords')
# nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
from collections import Counter

from gensim.corpora.dictionary import Dictionary

# import plotly
# import plotly.plotly as py
# import plotly.offline as offline
# import plotly.graph_objs as go
from nltk.tokenize import word_tokenize


['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [26]:
# read in data
# ----------------------------------
lines = pd.read_csv("/Users/dan/Documents/talk/the_office_area_plot/data/the_office_lines.csv", index_col=None)
lines.head()


,id,season,episode,scene,line_text,speaker,deleted
0,1,1,1,1,All right Jim. Your quarterlies look very good...,Michael,False
1,2,1,1,1,"Oh, I told you. I couldn't close it. So...",Jim,False
2,3,1,1,1,So you've come to the master for guidance? Is ...,Michael,False
3,4,1,1,1,"Actually, you called me in here, but yeah.",Jim,False
4,5,1,1,1,"All right. Well, let me show you how it's done.",Michael,False


In [27]:
# Convert season to string data type
# ----------------------------------
lines['season'] = lines['season'].astype(str)
lines.dtypes


id            int64
season       object
episode       int64
scene         int64
line_text    object
speaker      object
deleted        bool
dtype: object

In [28]:
# Word Tokenize
# --------
def word_token_lines(row):
    '''text preprocessing function

    1. remove stage instructions in brackets
    2. trim white space
    3. tokenize
    4. remove special characters
    5. remove stopwords
    6. stem all words

    arg:
        row of a df

    return:
        clean, tokenized row
    '''

    # remove stage instructions in brackets
    no_brackets = re.sub("[\(\[].*?[\)\]]", "", row.line_text)

    # trim white space
    no_white = no_brackets.strip()

    # tokenize words
    word_tokens = nltk.word_tokenize(no_white)

    # lowercase all words
    lower_tokens = [token.lower() for token in word_tokens]

    # remove special characters
    alpha_only = [token for token in lower_tokens
                    if token.isalpha()]

    # remove i, but, you, as, etc. (stopwords)
    no_stops = [token for token in alpha_only
                    if token not in stopwords.words('english')]

    # stem all words
    wordnet_lemmatizer = WordNetLemmatizer()
    lemmatized = [wordnet_lemmatizer.lemmatize(token) for token in no_stops]

    return lemmatized


def word_token_loop(df):
    # for loop to tokenize each row
    df['word_token'] = df.apply(word_token_lines, axis=1)
    return df


lines_word_token = word_token_loop(lines)
lines_word_token.head()

,id,season,episode,scene,line_text,speaker,deleted,word_token
0,1,1,1,1,All right Jim. Your quarterlies look very good...,Michael,False,"[right, jim, quarterly, look, good, thing, lib..."
1,2,1,1,1,"Oh, I told you. I couldn't close it. So...",Jim,False,"[oh, told, could, close]"
2,3,1,1,1,So you've come to the master for guidance? Is ...,Michael,False,"[come, master, guidance, saying, grasshopper]"
3,4,1,1,1,"Actually, you called me in here, but yeah.",Jim,False,"[actually, called, yeah]"
4,5,1,1,1,"All right. Well, let me show you how it's done.",Michael,False,"[right, well, let, show, done]"


In [29]:
# Speaker Tokenize
# --------
def word_token_speaker(row):
    '''text preprocessing function

    1. remove stage instructions in brackets
    2. trim white space
    3. tokenize
    4. remove special characters

    arg:
        row of a df

    return:
        clean, tokenized row
    '''

    # remove stage instructions in brackets
    no_brackets = re.sub("[\(\[].*?[\)\]]", "", row.speaker)

    # trim white space
    no_white = no_brackets.strip()

    # tokenize words
    word_tokens = nltk.word_tokenize(no_white)

    # lowercase all words
    lower_tokens = [token.lower() for token in word_tokens]

    # remove special characters
    alpha_only = [token for token in lower_tokens
                    if token.isalpha()]

    return alpha_only


def word_speaker_loop(df):
    # for loop to tokenize each row
    df['word_token_speaker'] = df.apply(word_token_speaker, axis=1)
    return df


lines_word_speaker = word_speaker_loop(lines_word_token)
lines_word_speaker.head()

,id,season,episode,scene,line_text,speaker,deleted,word_token,word_token_speaker
0,1,1,1,1,All right Jim. Your quarterlies look very good...,Michael,False,"[right, jim, quarterly, look, good, thing, lib...",[michael]
1,2,1,1,1,"Oh, I told you. I couldn't close it. So...",Jim,False,"[oh, told, could, close]",[jim]
2,3,1,1,1,So you've come to the master for guidance? Is ...,Michael,False,"[come, master, guidance, saying, grasshopper]",[michael]
3,4,1,1,1,"Actually, you called me in here, but yeah.",Jim,False,"[actually, called, yeah]",[jim]
4,5,1,1,1,"All right. Well, let me show you how it's done.",Michael,False,"[right, well, let, show, done]",[michael]


In [30]:
# lines_word_token['word_token'].head()

# i think onion to convert this1 column into a flat lreverseist and then the bag of words counter will work

# Create the bag-of-words: bow
# bow = Counter(lines_word_token['word_token'].to_list())
# stacked_list = lines_word_token['word_token'].head(1000).to_list()
# print(stacked_list)
# flat_list = [sentence for sub_list in stacked_list for sentence in sub_list]
# print(flat_list)

# bow = Counter(flat_list)
# Print the 10 most common tokens
# print(bow.most_common(10))


In [31]:

# lines_word_token['word_token'] = lines_word_token['word_token'].astype(str)
# lines_word_token['word_token_speaker'] = lines_word_token['word_token_speaker'].astype(str)
# grouped_lines = lines_word_token.groupby(['word_token_speaker'])['word_token'].apply(','.join).reset_index()
# lines_word_token.dtypes

In [32]:
# grouped_lines.to_csv('speaker_group_by.csv', index=False)
# lines_word_token.to_csv('lines_no_group.csv', index=False)

In [50]:
# grouped_lines['speaker'] = grouped_lines['value'].str[0]
lines_word_speaker['word_token_speaker'] = lines_word_speaker['word_token_speaker'].astype(str)
michael = lines_word_speaker[lines_word_speaker['speaker'] == 'Michael']
michael_list = michael['word_token'].to_list()
#print(michael_list)
# print(michael)
# michael.head()
# lines_word_speaker.head()
# michael = grouped_lines[grouped_lines['word_token_speaker'] == "['michael']"]
# michael['word_token'] = michael['word_token'].str.get(0)
# stacked_list = michael['word_token'].to_list()
# double_stacked = stacked_list.to_list()

michael_flat_list = [sentence for sub_list in michael_list for sentence in sub_list]
print(michael_flat_list)
# michael.to_csv('michael.csv', index=False)
# with open('your_file.txt', 'w') as f:
#     for item in stacked_list:
#         f.writelines(item)
# michael.head()

['right', 'jim', 'quarterly', 'look', 'good', 'thing', 'library', 'come', 'master', 'guidance', 'saying', 'grasshopper', 'right', 'well', 'let', 'show', 'done', 'yes', 'like', 'speak', 'office', 'manager', 'please', 'yes', 'hello', 'michael', 'scott', 'regional', 'manager', 'dunder', 'mifflin', 'paper', 'product', 'wanted', 'talk', 'right', 'done', 'deal', 'thank', 'much', 'sir', 'gentleman', 'scholar', 'oh', 'sorry', 'ok', 'sorry', 'mistake', 'woman', 'talking', 'low', 'voice', 'probably', 'smoker', 'way', 'done', 'uh', 'dunder', 'mifflin', 'year', 'last', 'four', 'regional', 'manager', 'want', 'come', 'see', 'entire', 'floor', 'kingdom', 'far', 'eye', 'see', 'receptionist', 'pam', 'pam', 'pam', 'beesly', 'pam', 'u', 'forever', 'right', 'pam', 'think', 'cute', 'seen', 'couple', 'year', 'ago', 'message', 'oh', 'pam', 'corporate', 'many', 'time', 'told', 'special', 'filing', 'cabinet', 'thing', 'corporate', 'called', 'wastepaper', 'basket', 'look', 'look', 'face', 'people', 'say', 'best

In [34]:
# from gensim.corpora.dictionary import Dictionary

# dictionary = Dictionary(grouped_lines)